In [10]:
# 1. Post 중 자신에게 포함된 Comment들 중에
# content 속성(column)에 '좋아요' 이라는 문자열이 들어가는 경우인 Post 목록
#     - 일부 Post가 '좋아요' 이라는 내용을 포함하는 댓글을 가져야 함
# 1-1. Post 중 태그가 없는 경우인 Post 목록

In [18]:
Post.objects.filter(
    postcomment__content__contains='좋아요'
).values('pk')

<QuerySet [{'pk': 6}, {'pk': 6}, {'pk': 6}]>

In [19]:
Post.objects.filter(
    tags__isnull=True
).values_list('pk', flat=True).order_by('-pk')

<QuerySet [8, 6, 4, 3, 2, 1]>

In [20]:
# 2. Post 중 자신에게 포함된 Comment들 중에
# Post의 author와 Comment의 author가 다른 경우인 Comment가 존재하는 Post 목록
#     > 댓글은 존재하나, 작성자가 작성한 댓글은 없는 경우인 Post 목록
#     - Post 작성자 외 다른 사용자가 댓들을 단 경우가 존재해야 함
#     - Django의 F Expression 사용

In [21]:
Post.objects.exclude(
    postcomment__isnull=True
).exclude(
    postcomment__author=F('author')
).values_list('pk', flat=True).order_by('-pk')

<QuerySet []>

In [22]:
# 3. HashTag 중 자신에게 연결된 Comment의 Post의 author의 pk가 1인 경우인 HashTag 목록
#     - 없다면 author의 pk를 바꿔봄

In [23]:
Tag.objects.filter(
    posts__author__pk=1
).values_list('posts', flat=True).distinct()

<QuerySet [5, 7, 9, 10, 11, 12, 13]>

In [24]:
# 4. Post 중 자신의 좋아요 개수가 1개 이상인(좋아요가 존재하는) Post 목록
#     - Field lookup의 'isnull' 항목 참조

In [25]:
Post.objects.filter(
    postlike__isnull=False
)

<QuerySet [<Post: practice | 2019-12-24 02:33:09.777656+00:00>, <Post: practice | 2019-12-24 02:33:09.777656+00:00>, <Post: practice | 2020-01-08 07:17:44.038017+00:00>, <Post: practice | 2020-01-08 08:36:40.039094+00:00>]>

In [26]:
Post.objects.exclude(
    postlike__isnull=True
).values_list('pk', flat=True)

<QuerySet [1, 3, 6]>

In [27]:
# 5. Comment 중 자신의 Post의 좋아요 개수가 1개 이상인 Comment 목록
#     - Field lookup의 'isnull' 항목 참조

In [33]:
PostComment.objects.filter(
    post__postlike__isnull=False
).values_list('pk', flat=True).distinct()

<QuerySet [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16]>

In [32]:
PostComment.objects.filter(
    post__postlike__isnull=False
).values_list('pk', 'post', 'post__postlike', 'post__postlike__user')

<QuerySet [(1, 6, 24, 1), (2, 6, 24, 1), (3, 6, 24, 1), (4, 6, 24, 1), (5, 6, 24, 1), (6, 6, 24, 1), (8, 6, 24, 1), (9, 6, 24, 1), (10, 6, 24, 1), (11, 6, 24, 1), (12, 6, 24, 1), (13, 6, 24, 1), (15, 6, 24, 1), (16, 6, 24, 1)]>

In [29]:
# 6. Comment 중 자신의 Post에 속하는 가장 최근 PostLike가 1주일 이내인 Comment 목록
#     - from django.utils import timezone
#     - 현재 시간은 timezone.now()로 가져올 수 있음

In [36]:
from datetime import timedelta

now = timezone.now()
a_week_ago = now - timedelta(days=7)

Post.objects.filter(
    postlike__created__gte=a_week_ago
).values('pk')

<QuerySet [{'pk': 6}, {'pk': 3}]>

In [43]:
# Comment 중 자신의 Post에 속하는 가장 최근 PostLike가 3일 이후인 Comment 목록
Post.objects.filter(
    postlike__created__lt=a_week_ago
).values('pk', 'postlike__user')

<QuerySet [{'pk': 1, 'postlike__user': 2}, {'pk': 1, 'postlike__user': 8}, {'pk': 6, 'postlike__user': 1}, {'pk': 3, 'postlike__user': 1}]>

In [44]:
PostComment.objects.filter(
    post__postlike__created__lt=a_week_ago
).values('pk', 'post')

<QuerySet [{'pk': 1, 'post': 6}, {'pk': 2, 'post': 6}, {'pk': 3, 'post': 6}, {'pk': 4, 'post': 6}, {'pk': 5, 'post': 6}, {'pk': 6, 'post': 6}, {'pk': 8, 'post': 6}, {'pk': 9, 'post': 6}, {'pk': 10, 'post': 6}, {'pk': 11, 'post': 6}, {'pk': 12, 'post': 6}, {'pk': 13, 'post': 6}, {'pk': 15, 'post': 6}, {'pk': 16, 'post': 6}]>

In [51]:
a_week_ago = now - timedelta(hours=2)

Post.objects.filter(
    postlike__created__lt=a_week_ago
).values('pk', 'postlike__user')

for item in PostComment.objects.filter(
    post__postlike__created__lt=a_week_ago
).values('pk', 'post', 'post__postlike__created').order_by('-post__pk').filter(
    post__pk=21
):
    print(item)

In [ ]:
# 7. 각각의 Post에 댓글이 몇 개 달렸는지 QuerySet.values()로 꺼내보기
#     Django annotate / aggregate
#     QueyrSetAPI의 Aggregation functions의 Count()를 사용해야 함
#         -> QueyrSet.count()와 다름
#     Post.objects.annotate(<무언가>).values(<왼쪽에서한걸활용>)

In [55]:
post = Post.objects.last()
post.postcomment_set.count()

0

In [56]:
Post.objects.annotate(
    comment_count=Count('postcomment')
).values_list('pk', 'comment_count')

<QuerySet [(1, 0), (2, 0), (3, 0), (4, 0), (5, 1), (6, 14), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0)]>

In [ ]:
# 8. 댓글이 3개 이상인 Post 목록만 가져오기
#     Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)

In [57]:
Post.objects.annotate(
    comment_count=Count('postcomment')
).filter(
    comment_count__gte=3
).values_list('pk', flat=True)

<QuerySet [6]>

In [ ]:
# 9. 댓글이 3개 이상이며, 댓글 개수가 좋아요 개수보다 많은 Post 목록만 가져오기
#     Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)

In [60]:
Post.objects.annotate(
    comment_count=Count('postcomment'),
    like_count=Count('postlike')
).filter(
    comment_count__gte=3
#     comment_count__gt=F('like_count')
).values_list('pk', flat=True)

<QuerySet [6]>